In [19]:
# Update json files 
# Daily Looping
from backtesting import Backtest, Strategy
import yfinance as yf
import json
import pandas as pd
import datetime
import pandas_ta as ta
import tqdm
import subprocess
route = "../Database/"
filemap = {
    "NYSE"  : route + "../Database/zackRanks_NYSE.json", # 1352
    "NASDAQ": route + '../Database/zackRanks_NASDAQ.json', # small:1255; mid:330; large:72
    "SP500" : route + "../Database/zackRanks_SP500.json", # small: 1352
   "Yuanta": route + "zackRanks_yuanta.json",
   "buy": route + "buy.json",
   "sell": route + "sell.json",
   "portfolio": route + "zackRanks_portfolio.json"
}

def mains(today=0, loops = ["NYSE", "Yuanta", "NASDAQ", "SP500"]):
    for k in loops:
        print("*"*20, k, "*"*20)
        with open(filemap[k], 'r') as f:
            python_dict = json.load(f)
            a = list(python_dict.keys())
            with tqdm.tqdm(total=len(a)) as pbar:
                for ticker in a:
                    command = "zacks-api " + ticker
                    result = subprocess.check_output(command, shell=True)
                    dict = eval(result.decode(("utf-8")))
                    tick = dict["ticker"]
                    rank = dict["zacksRank"]
                    if today == 0:
                        dat = dict["updatedAt"].split("T")[0]
                    else:
                        dat = datetime.datetime.now() - datetime.timedelta(days=today)
                    python_dict[tick][dat] = rank
                    pbar.update(1)
        with open(filemap[k], 'w') as f:
            json.dump(python_dict, f, indent=4)


loops = ["NYSE", "Yuanta", "NASDAQ", "SP500", "portfolio"]
loop = ["SP500"]
# SP500: 
# today: default_day = 0
mains(today=0, loops=loop)

******************** SP500 ********************


100%|██████████| 499/499 [10:13<00:00,  1.23s/it]


In [ ]:
# update Tickers/{}.csv
import datetime
import pandas as pd
import yfinance as yf
route = "../Database/"
filemap = {
    "NYSE"  : route + "../Database/zackRanks_NYSE.json", # 1352
    "NASDAQ": route + '../Database/zackRanks_NASDAQ.json', # small:1255; mid:330; large:72
    "SP500" : route + "../Database/zackRanks_SP500.json", # small: 1352
   "Yuanta": route + "zackRanks_yuanta.json",
   "buy": route + "buy.json",
   "sell": route + "sell.json",
   "portfolio": route + "zackRanks_portfolio.json"
}
def daily_add_datetime_minus_1(loops=list(filemap.keys())):
    for kys in loops:
        with open(filemap[kys], "r") as f:
            python_dict = json.load(f)
            a = list(python_dict.keys())
        i = 0
        for ticker in a:
            df = pd.read_csv(f"../Database/Tickers/{ticker}.csv", index_col="Date")
            start = datetime.datetime.now()-datetime.timedelta(days=1)
            ftime = start.strftime("%Y-%m-%d")
            try:
                b = df.loc[ftime]
            except KeyError:
                a = yf.download("A", start=start)
                a.index = [ftime]
                df = pd.concat([df, a])
                df.index.name = "Date"
                df.to_csv("../Database/Tickers/A.csv", index="Date")
                if i == 0:
                    print(f"added on {kys}")
                    i += 1
daily_add_datetime_minus_1()                

In [ ]:
# initialization of Tickers/{}.csv
import datetime
import pandas as pd
import yfinance as yf
route = "../Database/"
filemap = {
    "NYSE"  : route + "../Database/zackRanks_NYSE.json", # 1352
    "NASDAQ": route + '../Database/zackRanks_NASDAQ.json', # small:1255; mid:330; large:72
    "SP500" : route + "../Database/zackRanks_SP500.json", # small: 1352
   "Yuanta": route + "zackRanks_yuanta.json",
   "buy": route + "buy.json",
   "sell": route + "sell.json",
   "portfolio": route + "zackRanks_portfolio.json"
}
def database_initialization():
    for kys in ["NYSE", "NASDAQ", "SP500"]:
        with open(filemap[kys], "r") as f:
            python_dict = json.load(f)
            a = list(python_dict.keys())
        a = list(python_dict.keys())
        for ticker in a:
            print(ticker)
            df = yf.download(ticker, progress=False)
            df.to_csv(f"../Database/Tickers/{ticker}.csv")
        
daily_add_datetime_minus_1()                
